In [1]:
import sys
import pandas as pd
import numpy as np
import datetime
import os #Library module for .csv file check
from twitterscraper import query_tweets

INFO: {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; x64; fr; rv:1.9.2.13) Gecko/20101203 Firebird/3.6.13'}


In [2]:
#---------------------------------------------------------------------


#Get tweets without geolocation
def get_tweets(query, year, month, day): 
    tweets = {} #Instantiates a new empty dictionary
    count = 0 #Instantiates an index generator
    for tweet in query_tweets(query,begindate=datetime.date(year,month,day)):
        chirp = {} #Instantiates a new empty dictionary for each pulled tweet
        chirp['tweet_id'] = tweet.tweet_id
        chirp['username'] = tweet.username
        chirp['text'] = tweet.text
        chirp['tweet_date'] = tweet.timestamp
        chirp['search_term'] = query
        chirp['city'] = np.NaN #Fills columns with NaNs for data cleaning at a later point. These items are NaNs so that
        chirp['lat'] = np.NaN #They will be flagged in an EDA search for missing values, instead of being strings with no information
        chirp['long'] = np.NaN
        chirp['radius'] = np.NaN
        chirp['query_start'] = pd.to_datetime(f"{year}/{month}/{day}") #Generates date as datetime object
        tweets.update({count : chirp}) #Sets count value to tweets keys, so that index is automatically generated 
        count += 1 #increments index up by 1
    return tweets
#--------------------------------------------------------------------

#Get tweets with geolocation
def get_tweets_geoloc(query, city, lat, long, radius, year, month, day): #Geolocation parameters defined by user in master function or dictionary
    tweets = {}
    count = 0
    for tweet in query_tweets(f"{query}, geocode:{lat},{long},{radius}",begindate=datetime.date(year,month,day)):
        chirp = {} #Generates tweet dictionary by calling on generated 'tweet' object attributes
        chirp['tweet_id'] = tweet.tweet_id
        chirp['username'] = tweet.username
        chirp['text'] = tweet.text
        chirp['tweet_date'] = tweet.timestamp
        chirp['search_term'] = query
        chirp['city'] = city
        chirp['lat'] = lat
        chirp['long'] = long
        chirp['radius'] = radius
        chirp['query_start'] = pd.to_datetime(f"{year}/{month}/{day}") #Generates date as datetime object
        tweets.update({count : chirp}) #Sets count value to tweets keys, so that index is automatically generated 
        count += 1 #increments index up by 1
    return tweets
#--------------------------------------------------------------------

#Generate dataframe from "tweets" dictionary generated after each query
def make_dataframe(dictionary):
    df = pd.DataFrame.from_dict(dictionary, orient='index') #Creates a dataframe from the input dictionary 'tweets' later in function
    return df #returns a temporary dataframe for concatenation later
#--------------------------------------------------------------------

#Query function using custom parameters
def get_query_dataframe_cp(custom_params):
    query_df = pd.DataFrame() #instantiate an empty dataframe
    for key in custom_params.keys(): #Generates a new query dataframe for each city used in the parameter dictionary
        for query in custom_params[key]['queries']: #Runs a unique query for each unique term in the query key
            tweets = get_tweets_geoloc(query,custom_params[key]['city'],custom_params[key]['lat'],custom_params[key]['long'],
                                       custom_params[key]['radius'],custom_params[key]['start_year'],custom_params[key]['start_month'],
                                       custom_params[key]['start_day'])
            df = make_dataframe(tweets) #creates temporary dataframe from independent query
            query_df = pd.concat([query_df,df],ignore_index = True) #concatenates temporary dataframe 'df' to master query dataframe 
    return query_df
#Credit Danielle Medellin for the majority of this code block section
#---------------------------------------------------------------

#Query function with geolocation but no custom parameters
def get_query_dataframe_geo(list_of_queries,city,lat,long,radius,year,month,day):
    query_df = pd.DataFrame()
    for query in list_of_queries: #Runs the same functions as custom parameters code
            tweets = get_tweets_geoloc(query,city,lat,long,radius,year,month,day)
            df = make_dataframe(tweets)
            query_df = pd.concat([query_df,df],ignore_index = True)
    return query_df
#-------------------------------------------------------------------

#Query function with no custom anything
def get_query_dataframe(list_of_queries,year,month,day):
    query_df = pd.DataFrame()
    for query in list_of_queries:
            tweets = get_tweets(query,year,month,day)
            df = make_dataframe(tweets)
            query_df = pd.concat([query_df,df],ignore_index = True)
    return query_df
#------------------------------------------------------------------

#Master function
def get_dataset():   
    #Main function switches
    custom_params_switch = input("Are you using a custom parameter dictionary? (y/n)") #Enables or disables custom dictionary use in query parameter switch block
    if str.lower(custom_params_switch) != 'y': #If the user is not using parameters
        
        #Geolocation parameter configuration
        geo_switch = input("Are you using geolocation? (y/n)") #If no custom dictionary, enables or disables geolocation in query parameter switch block
        if str.lower(geo_switch) == 'y':
            lat = float(input("Input Latitude:")) #Converts string input latitude to float value
            long = float(input("Input Longitude:"))
            city = input("Input city or neighborhood corresponding to coordinates:") #Allows filling of city values
            radius = input("Input radius and unit:")
            
        #Date parameter configuration
        use_custom_start_date = input("Using custom query start? (y/n)") #Enables or disables custom date parameter
        if str.lower(use_custom_start_date) == 'y':
            year = int(input("Input start year YYYY:"))
            month = int(input("Input start month MM:"))
            day = int(input("Input start day DD:"))
        else: #Sets default date
            
            #Default date parameters. Users can change defaults here
            year = 2019
            month = 12
            day = 1
 
    #CSV export parameter configuration
    export_csv_switch = input("Do you want to export the final dataframe to csv? (y/n)")
    if str.lower(export_csv_switch) == 'y': #Enables CSV switch block later if yes as well
        custom_csv_name = input("Input CSV export file name:") #Prompts user for file name
        if os.path.exists(f'datasets/{custom_csv_name}.csv') == True: #Check if file has already been created. If yes, prompt user to overwrite or make new file.
            overwrite_check = input ("File already exists--do you want to overwrite? (y/n)")
            if str.lower(overwrite_check) != 'y': #If user uses any value except y, the function will not proceed
                new_csv_name = custom_csv_name #creates new_csv_name variable = to old name
                while new_csv_name == custom_csv_name: #continues to reject file name until a unique name is created
                    new_csv_name = input("Input new output file name:")
                custom_csv_name = new_csv_name #Sets the file name to the new user input
    
    #Query parameter switch block
    if str.lower(custom_params_switch) == 'y':
        dataset = get_query_dataframe_cp(custom_params)
    else:
        if str.lower(geo_switch) == 'y':
            dataset = get_query_dataframe_geo(query_list,city,lat,long,radius,year,month,day)
        else:
            dataset = get_query_dataframe(query_list,year,month,day)
    
    #CSV export switch block
    if str.lower(export_csv_switch) == 'y':
        if os.path.exists('datasets') != True:
            os.mkdir('datasets')
        dataset.to_csv(f"./datasets/{custom_csv_name}.csv", index = False) #write csv to datasets folder
        print(f"Export complete, scraped {len(dataset.index)} tweets") #Prints completion statement including total tweets scraped       
    else:
        return dataset




#This is a sample list of terms
california_query_list = [
    'power outage',
    'power out',
    'transformer AND power',
    'power cut',
    'power failure',
    'power interruption',
    'service interruption',
    'lights out',
    'electric out',
    'power off',
    'electric off',
    'lost power',
    '#poweroutage',
    #los angeles and san jose
    'Pacific Gas and Electric AND power',
    'Pacific Gas and Electric AND outage',
    'Pacific Gas & Electric AND outage',
    'PG&E AND power',
    'PG&E AND outage',
    #san diego
    'SDG&E AND power',
    'SDG&E AND outage',
    'SDGE AND outage',
    'SDGE AND power',
    'San Diego Gas Electric AND power',
    'San Diego Gas & Electric AND power',
    'San Diego Gas & Electric AND outage'
]

#--------------------------------------------------------------------

#This is a sample filled-out custom parameter dictionary.
custom_params = {'San Jose':{'city' : 'San Jose',
                         'lat'  : 37.332300,
                         'long' : - 121.853394,
                         'radius': '10mi',
                         'queries' : california_query_list,
                         'start_year': 2016,
                         'start_month': 1,
                         'start_day': 1},
              'Los Angeles': {'city' : 'Los Angeles',
                          'lat'  : 34.052235,
                          'long' : -118.243683,
                          'radius' : '15mi',
                          'queries': california_query_list,
                          'start_year': 2016,                         #query_list can be used as the values for "queries"
                          'start_month': 1,
                          'start_day': 1},
              'San Diego':{'city' : 'San Diego',
                         'lat'  : 32.801615,
                         'long' : -117.07058,
                         'radius': '10mi',
                         'queries' : california_query_list,
                         'start_year': 2016,                         #query_list can be used as the values for "queries"
                         'start_month': 1,
                         'start_day': 1}
        }

#Credit to Danielle Medellin, DSI11-NYC for assisting in implementation of custom parameter dictionary support
#--------------------------------------------------------------------



In [3]:
get_dataset()

Are you using a custom parameter dictionary? (y/n)y
Do you want to export the final dataframe to csv? (y/n)y
Input CSV export file name:california_dataset


INFO: queries: ['power outage, geocode:37.3323,-121.853394,10mi since:2016-01-01 until:2016-03-20', 'power outage, geocode:37.3323,-121.853394,10mi since:2016-03-20 until:2016-06-08', 'power outage, geocode:37.3323,-121.853394,10mi since:2016-06-08 until:2016-08-26', 'power outage, geocode:37.3323,-121.853394,10mi since:2016-08-26 until:2016-11-14', 'power outage, geocode:37.3323,-121.853394,10mi since:2016-11-14 until:2017-02-02', 'power outage, geocode:37.3323,-121.853394,10mi since:2017-02-02 until:2017-04-22', 'power outage, geocode:37.3323,-121.853394,10mi since:2017-04-22 until:2017-07-11', 'power outage, geocode:37.3323,-121.853394,10mi since:2017-07-11 until:2017-09-28', 'power outage, geocode:37.3323,-121.853394,10mi since:2017-09-28 until:2017-12-17', 'power outage, geocode:37.3323,-121.853394,10mi since:2017-12-17 until:2018-03-07', 'power outage, geocode:37.3323,-121.853394,10mi since:2018-03-07 until:2018-05-25', 'power outage, geocode:37.3323,-121.853394,10mi since:2018-0

INFO: Using proxy 180.252.181.2:80
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1105718177059598341-1105718177059598341&q=power%20outage,%20geocode%3A37.3323,-121.853394,10mi%20since%3A2019-01-19%20until%3A2019-04-09&l=
INFO: Using proxy 180.252.181.2:80
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-855585710270586881-876961200906620929&q=power%20outage,%20geocode%3A37.3323,-121.853394,10mi%20since%3A2017-04-22%20until%3A2017-07-11&l=
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1065318920200966145-1066431068855664640&q=power%20outage,%20geocode%3A37.3323,-

INFO: Using proxy 1.20.97.238:31769
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaMwKjxv9uHrxYWgMCk6fDL6_AWEjUAFQAlAFUAFQAA&q=power%20outage,%20geocode%3A37.3323,-121.853394,10mi%20since%3A2016-11-14%20until%3A2017-02-02&l=
INFO: Using proxy 1.20.97.238:31769
INFO: Retrying... (Attempts left: 49)
INFO: Scraping tweets from https://twitter.com/search?f=tweets&vertical=default&q=power%20outage,%20geocode%3A37.3323,-121.853394,10mi%20since%3A2017-07-11%20until%3A2017-09-28&l=
INFO: Using proxy 1.20.97.238:31769
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAwLXRhe316xwWgMC10YXt9escEjUAFQAlAFUAFQAA&q=power%20outage,%20geocode%3A37.3323,-121.853394,10mi%20since%3A2018-08-13%20until%3A2018-10-

INFO: Using proxy 125.27.251.57:41425
INFO: Using proxy 125.27.251.57:41425
INFO: Using proxy 125.27.251.57:41425
INFO: Twitter returned : 'has_more_items' 
INFO: Got 28 tweets for power%20outage,%20geocode%3A37.3323,-121.853394,10mi%20since%3A2020-02-21%20until%3A2020-05-11.
INFO: Got 176 tweets (28 new).
INFO: Retrying... (Attempts left: 47)
INFO: Scraping tweets from https://twitter.com/search?f=tweets&vertical=default&q=power%20outage,%20geocode%3A37.3323,-121.853394,10mi%20since%3A2017-07-11%20until%3A2017-09-28&l=
INFO: Using proxy 192.41.71.204:3128
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=thGAVUV0VFVBaAgKfpzs_F6CAWgIC3yZHTgqwhEjUAFQAlAFUAFQAA&q=power%20outage,%20geocode%3A37.3323,-121.853394,10mi%20since%3A2019-09-15%20until%3A2019-12-03&l=
INFO: Using proxy 125.27.251.57:41425
INFO: Retrying... (Attempts left: 47)
INFO: Scraping tweets

...

INFO: Using proxy 52.179.231.206:80
INFO: Retrying... (Attempts left: 8)
INFO: Scraping tweets from https://twitter.com/search?f=tweets&vertical=default&q=electric%20out,%20geocode%3A37.3323,-121.853394,10mi%20since%3A2016-06-08%20until%3A2016-08-26&l=
INFO: Using proxy 63.82.52.254:8080
INFO: Retrying... (Attempts left: 13)
INFO: Scraping tweets from https://twitter.com/search?f=tweets&vertical=default&q=electric%20out,%20geocode%3A37.3323,-121.853394,10mi%20since%3A2017-09-28%20until%3A2017-12-17&l=
INFO: Using proxy 185.189.211.70:8080
INFO: Retrying... (Attempts left: 7)
INFO: Scraping tweets from https://twitter.com/search?f=tweets&vertical=default&q=electric%20out,%20geocode%3A37.3323,-121.853394,10mi%20since%3A2017-12-17%20until%3A2018-03-07&l=
INFO: Retrying... (Attempts left: 5)
INFO: Using proxy 151.232.72.14:80
INFO: Scraping tweets from https://twitter.com/search?f=tweets&vertical=default&q=electric%20out,%20geocode%3A37.3323,-121.853394,10mi%20since%3A2018-08-13%20until%3A

INFO: Using proxy 12.139.101.98:80
INFO: Retrying... (Attempts left: 9)
INFO: Scraping tweets from https://twitter.com/search?f=tweets&vertical=default&q=electric%20out,%20geocode%3A37.3323,-121.853394,10mi%20since%3A2016-11-14%20until%3A2017-02-02&l=
INFO: Using proxy 12.139.101.98:80
INFO: Retrying... (Attempts left: 4)
INFO: Scraping tweets from https://twitter.com/search?f=tweets&vertical=default&q=electric%20out,%20geocode%3A37.3323,-121.853394,10mi%20since%3A2018-05-25%20until%3A2018-08-13&l=
INFO: Using proxy 60.51.170.27:80
INFO: Retrying... (Attempts left: 9)
INFO: Retrying... (Attempts left: 11)
INFO: Retrying... (Attempts left: 4)
INFO: Scraping tweets from https://twitter.com/search?f=tweets&vertical=default&q=electric%20out,%20geocode%3A37.3323,-121.853394,10mi%20since%3A2016-08-26%20until%3A2016-11-14&l=
INFO: Scraping tweets from https://twitter.com/search?f=tweets&vertical=default&q=electric%20out,%20geocode%3A37.3323,-121.853394,10mi%20since%3A2016-06-08%20until%3A2016

INFO: Scraping tweets from https://twitter.com/search?f=tweets&vertical=default&q=electric%20out,%20geocode%3A37.3323,-121.853394,10mi%20since%3A2016-11-14%20until%3A2017-02-02&l=
INFO: Using proxy 42.3.51.34:80
INFO: Got 0 tweets for electric%20out,%20geocode%3A37.3323,-121.853394,10mi%20since%3A2018-05-25%20until%3A2018-08-13.
INFO: Got 70 tweets (0 new).
INFO: Retrying... (Attempts left: 7)
INFO: Scraping tweets from https://twitter.com/search?f=tweets&vertical=default&q=electric%20out,%20geocode%3A37.3323,-121.853394,10mi%20since%3A2016-08-26%20until%3A2016-11-14&l=
INFO: Got 0 tweets for electric%20out,%20geocode%3A37.3323,-121.853394,10mi%20since%3A2016-06-08%20until%3A2016-08-26.
INFO: Using proxy 151.232.72.14:80
INFO: Got 70 tweets (0 new).
INFO: Retrying... (Attempts left: 5)
INFO: Retrying... (Attempts left: 5)
INFO: Scraping tweets from https://twitter.com/search?f=tweets&vertical=default&q=electric%20out,%20geocode%3A37.3323,-121.853394,10mi%20since%3A2017-02-02%20until%3A

INFO: Querying power off, geocode:37.3323,-121.853394,10mi since:2016-11-14 until:2017-02-02
INFO: Querying power off, geocode:37.3323,-121.853394,10mi since:2016-08-26 until:2016-11-14
INFO: Querying power off, geocode:37.3323,-121.853394,10mi since:2016-06-08 until:2016-08-26
INFO: Querying power off, geocode:37.3323,-121.853394,10mi since:2016-01-01 until:2016-03-20
INFO: Querying power off, geocode:37.3323,-121.853394,10mi since:2017-09-28 until:2017-12-17
INFO: Querying power off, geocode:37.3323,-121.853394,10mi since:2018-03-07 until:2018-05-25
INFO: Querying power off, geocode:37.3323,-121.853394,10mi since:2017-12-17 until:2018-03-07
INFO: Querying power off, geocode:37.3323,-121.853394,10mi since:2018-05-25 until:2018-08-13
INFO: Querying power off, geocode:37.3323,-121.853394,10mi since:2017-04-22 until:2017-07-11
INFO: Querying power off, geocode:37.3323,-121.853394,10mi since:2016-03-20 until:2016-06-08
INFO: Querying power off, geocode:37.3323,-121.853394,10mi since:2018-

Exception: Traceback (most recent call last):
  File "/opt/anaconda3/envs/DSI/lib/python3.7/site-packages/billiard/pool.py", line 1267, in mark_as_worker_lost
    human_status(exitcode)),
billiard.exceptions.WorkerLostError: Worker exited prematurely: signal 11 (SIGSEGV).
